# Paddy disease classification
Kaggle competition
https://www.kaggle.com/competitions/paddy-disease-classification

### Setting up env and downloading data from kaggle

In [1]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
  colab_flag = True
else:
  colab_flag = False

if colab_flag == True:
   print("Running in Colab...")
   #Install Kaggle API
   ! pip install -q kaggle
   #Install Huggingface Datasets lib
   ! pip install -q datasets
   #Install Huggingface transformers & the sentencepiece
   ! pip install -q transformers[sentencepiece,torch]
   #Intall Timm for Pytorch Image Models
   ! pip install -q timm
   print("...Installed required dependencies")
else:
   print("Assuming running in local environment...")

Running in Colab...
...Installed required dependencies


In [2]:
if colab_flag == True:
  import json
  from google.colab import drive
  drive.mount('/content/drive/')

  # Set the file path
  file_path = '/content/drive/MyDrive/dtu/fastAI/04_NLP/kaggle_api.json'

  # Check if the file exists
  if os.path.exists(file_path):
      # Load the JSON file
      with open(file_path) as f:
          creds = json.load(f)
      print('Sucesfully set kaggle credentials')
  else:
      # Handle the case when the file does not exist
      creds = None  # or any other appropriate action you want to take
      print('Error: File not found, Credentials NOT set')
else:
    import json
    file_path = "../secrets/kaggle_api.json"
    # Check if the file exists
    if os.path.exists(file_path):
        # Load the JSON file
        with open(file_path) as f:
            creds = json.load(f)
        print('Sucesfully set kaggle credentials')
    else:
        # Handle the case when the file does not exist
        creds = None  # or any other appropriate action you want to take
        print('Error: File not found, Credentials NOT set')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Sucesfully set kaggle credentials


In [3]:
#using `pathlib.Path to work with paths in Python`
from pathlib import Path
#Set path for the titanic dataset
path = Path('paddy-disease-classification')

if path.exists():
  print('Data folder exists')
else:
  print('Data not detected, starting download')
  #Setup of kaggle credentials to use the API for downloading dataset
  cred_path = Path('~/.kaggle/kaggle.json').expanduser()
  if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(json.dumps(creds))
    #If not Json
    #cred_path.write_text(creds)
    cred_path.chmod(0o600)
  import zipfile,kaggle
  #Download data
  kaggle.api.competition_download_cli(str(path))
  #Unzip at path
  zipfile.ZipFile(f'{path}.zip').extractall(path)

#Verifying the the local content
!ls {path}

Data folder exists
sample_submission.csv  test_images  train.csv  train_images


In [4]:
import timm
from fastai.vision.all import *
set_seed(42)

train_path = path/'train_images'
train_files = get_image_files(train_path)

test_path = path/'test_images'
test_files = get_image_files(train_path)

# Testing GPU limitation
The goal of this notebook is to improve the performance by using larger models, but at this point GPU memory will grow into a limitation.

It's really helpful to be able to quickly try a few models and image sizes and find out what will run successfully. To make this quick, we can just grab a small subset of the data for running short epochs -- the memory use will still be the same, but it'll be much faster.

One easy way to do this is to simply pick a category with few files in it. Here's our options:


In [5]:
#Simply contains the filename of all the images in the given folder
test_files[0:3]

(#3) [Path('paddy-disease-classification/train_images/bacterial_leaf_blight/109910.jpg'),Path('paddy-disease-classification/train_images/bacterial_leaf_blight/103313.jpg'),Path('paddy-disease-classification/train_images/bacterial_leaf_blight/107254.jpg')]

In [6]:
df_test = pd.read_csv(path/'train.csv')
df_test.head()

,image_id,label,variety,age
0,100330.jpg,bacterial_leaf_blight,ADT45,45
1,100365.jpg,bacterial_leaf_blight,ADT45,45
2,100382.jpg,bacterial_leaf_blight,ADT45,45
3,100632.jpg,bacterial_leaf_blight,ADT45,45
4,101918.jpg,bacterial_leaf_blight,ADT45,45


In [7]:
df_test.label.value_counts()

normal                      1764
blast                       1738
hispa                       1594
dead_heart                  1442
tungro                      1088
brown_spot                   965
downy_mildew                 620
bacterial_leaf_blight        479
bacterial_leaf_streak        380
bacterial_panicle_blight     337
Name: label, dtype: int64

### Selecting just a small subset of the test data

In [8]:
train_path_subset = path/'train_images'/'bacterial_panicle_blight'

In [9]:
#Define a train function to test different models
def train_model(arch, aug_size, item=Resize(480, method='squish'), accum=1, finetune=True,epochs=12):
  dls = ImageDataLoaders.from_folder(train_path_subset,
                                        valid_pct=0.2,
                                        item_tfms=item,
                                        #default bs=64
                                        batch_tfms=aug_transforms(size=aug_size, min_scale=0.75), bs=64//accum)
  cbs = GradientAccumulation(64) if accum else []
  #cbs is one or a list of Callbacks to pass to the Learner. Callbacks are used for every tweak of the training loop.
  learn = vision_learner(dls,arch,metrics=error_rate, cbs=cbs).to_fp16()
  if finetune:
    learn.fine_tune(epochs, 0.01)
    return learn.tta(dl=dls.test_dl(test_files))
  else:
    learn.unfreeze()
    learn.fit_one_cycle(epochs, 0.01)

In [10]:
!pip install pynvml

import gc
def report_gpu():
  print(torch.cuda.list_gpu_processes())
  gc.collect()
  torch.cuda.empty_cache()

In [11]:
report_gpu()

GPU:0
no processes are running


In [12]:
import timm
from fastai.vision.all import *

train_model('convnext_small_in22k', 128, epochs=1, accum=1, finetune=False)
report_gpu()

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name convnext_small_in22k to current convnext_small.fb_in22k.
  model = create_fn(


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:12


GPU:0
process    1327826 uses     4254.000 MB GPU memory


In [13]:
train_model('convnext_small_in22k', 128, epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:03


GPU:0
process    1327826 uses     3194.000 MB GPU memory


In [14]:
train_model('convnext_small_in22k', 128, epochs=1, accum=4, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:10


GPU:0
process    1327826 uses     2658.000 MB GPU memory


## Larger models
Overview of models:

https://www.kaggle.com/code/jhoward/which-image-models-are-best

https://www.kaggle.com/code/jhoward/the-best-vision-models-for-fine-tuning

In [15]:
train_model('convnext_large_in22k', 224, epochs=1, accum=2, finetune=False)
report_gpu()

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name convnext_large_in22k to current convnext_large.fb_in22k.
  model = create_fn(


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:08


GPU:0
process    1327826 uses    11080.000 MB GPU memory


In [16]:
#Larger image augm
train_model('convnext_large_in22k', (320,240), epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:11


GPU:0
process    1327826 uses    14416.000 MB GPU memory


In [17]:
#Vit large
train_model('vit_large_patch16_224', 224, epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:08


GPU:0
process    1327826 uses    12510.000 MB GPU memory


Some problem when trying to run the Swin models...

"RuntimeError: running_mean should contain 14 elements not 3072"

In [18]:
#swin v2
#train_model('swin_large_patch4_window7_224.ms_in22k', 224, epochs=1, accum=2, finetune=False)
#report_gpu()

In [23]:
report_gpu()

#Beit
train_model('beitv2_large_patch16_224.in1k_ft_in22k_in1k', 224, epochs=1, accum=2, finetune=False)
report_gpu()


GPU:0
process    1327826 uses     1326.000 MB GPU memory


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:11


GPU:0
process    1327826 uses    14150.000 MB GPU memory


# Running the models


In [5]:
res = 640,480

models = {
    'convnext_large_in22k': {
        (Resize(res), (320,224)),},
    #Vit is transformer based and can only handle fixed input size, hence the first resize
    'vit_large_patch16_224': {
        (Resize(480, method='squish'), 224),
        (Resize(res), 224)},
     #Beit is transformer based and can only handle fixed input size, hence the first resize
    'beitv2_large_patch16_224.in1k_ft_in22k_in1k': {
        (Resize(480, method='squish'), 224),
        (Resize(res), 224)},
}

In [7]:
#redefine train using the train_path (not subset)
def train_model(arch, aug_size, item=Resize(480, method='squish'), accum=1, finetune=True,epochs=12):
  dls = ImageDataLoaders.from_folder(train_path,
                                        valid_pct=0.2,
                                        item_tfms=item,
                                        #default bs=64
                                        batch_tfms=aug_transforms(size=aug_size, min_scale=0.75), bs=64//accum)
  cbs = GradientAccumulation(64) if accum else []
  #cbs is one or a list of Callbacks to pass to the Learner. Callbacks are used for every tweak of the training loop.
  learn = vision_learner(dls,arch,metrics=error_rate, cbs=cbs).to_fp16()
  if finetune:
    learn.fine_tune(epochs, 0.01)
    return learn.tta(dl=dls.test_dl(test_files))
  else:
    learn.unfreeze()
    learn.fit_one_cycle(epochs, 0.01)

In [8]:
tta_res = []

for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        tta_res.append(train_model(arch, size, item=item, accum=2)) #, epochs=1))
        gc.collect()
        torch.cuda.empty_cache()

--- convnext_large_in22k
(320, 224)
Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name convnext_large_in22k to current convnext_large.fb_in22k.
  model = create_fn(


epoch,train_loss,valid_loss,error_rate,time
0,0.803613,0.452207,0.143200,03:30


epoch,train_loss,valid_loss,error_rate,time
0,0.367735,0.210611,0.066795,04:55
1,0.317118,0.290828,0.074964,04:55
2,0.265762,0.210918,0.062470,04:58
3,0.262109,0.235739,0.062951,04:55
4,0.170555,0.207167,0.054301,04:54
5,0.167588,0.166476,0.038924,04:53
6,0.090160,0.166624,0.037001,04:53
7,0.071090,0.136710,0.033638,04:54
8,0.039219,0.110290,0.020663,04:52
9,0.036758,0.091745,0.022585,04:52


NameError: ignored

## Ensembling

### Saving the model

In [ ]:
learn.save("/content/drive/MyDrive/dtu/fastAI/06_Paddy_vision/models")

In [ ]:
doc(Learner.save)